In [1]:
import requests
from datetime import datetime, timedelta
import pickle
import pandas as pd

In [2]:
key = 'f8599fa0624567b98d6293acc87489bb-e288ec05b46b6e3d0bc753e6a2fbab48'

In [3]:
def dataGenerator(typ, start, end):
    
    reqNumber = (datetime.strptime(end, "%Y-%m-%dT%H:%M:%SZ") -\
    datetime.strptime(start, "%Y-%m-%dT%H:%M:%SZ")).total_seconds()/60/20

    date = datetime.strptime(start, "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%SZ")

    for i in range(int(reqNumber)):
        
        
        yield requests.get(
                url = 'https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/'+typ,
                headers = {
                    'Authorization' : 'Bearer ' + key,
                },
                data = {
                    'time': date
                }
            ).json()

        date = (datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ") + timedelta(minutes=20)).strftime("%Y-%m-%dT%H:%M:%SZ")

In [4]:
def processRecord(response, typ):
    price = float(response[typ]['price'])
            
    estimatePrices = []
    estimatePrices.append(float(str(round(price, zerosCounter))))
    estimatePrices.append(round(round(price, zerosCounter) - float(numberToRound / (10 ** (zerosCounter + 1))), zerosCounter + 1))
    estimatePrices.append(round(round(price, zerosCounter) + float(numberToRound / (10 ** (zerosCounter + 1))), zerosCounter + 1))

    diff = 100
    chosenMiddlePriceIndex = 0
    for i, p in enumerate(estimatePrices):
        if abs(price - p) < diff:
            diff = abs(price - p)
            chosenMiddlePriceIndex = i

    middlePrice = estimatePrices[chosenMiddlePriceIndex]

    tmpBuckets = {}
    for bucket in response[typ]['buckets']:
        tmpBuckets[float(bucket['price'])] = {
            'l': bucket['longCountPercent'],
            's': bucket['shortCountPercent'],
        }
    pricesKeys = list(tmpBuckets.keys())

    row = [
        response[typ]['time'],
        response[typ]['price'],
        middlePrice,
    ]

    if middlePrice in pricesKeys:
        row.append(tmpBuckets[middlePrice]['l'])
        row.append(tmpBuckets[middlePrice]['s'])
    else:
        row.append(0)
        row.append(0)

    UpPrice = middlePrice
    DownPrice = middlePrice
    floatedPriceBucket = float(priceRange)
    for i in range(5):
        UpPrice = UpPrice + floatedPriceBucket
        if UpPrice in pricesKeys:
            row.append(tmpBuckets[UpPrice]['l'])
            row.append(tmpBuckets[UpPrice]['s'])
        else:
            row.append(0)
            row.append(0)

        DownPrice = DownPrice + floatedPriceBucket
        if DownPrice in pricesKeys:
            row.append(tmpBuckets[DownPrice]['l'])
            row.append(tmpBuckets[DownPrice]['s'])
        else:
            row.append(0)
            row.append(0)
            
    return row

In [5]:
def getInstrumentStats(response, typ):
    priceRange = response[typ]['bucketWidth']
    numberToRound = len(str(response[typ]['bucketWidth']).split('.')[1])

    zerosCounter = 0
    for number in str(priceRange).split('.')[1]:
        if number == '0':
            zerosCounter = zerosCounter + 1
        else:
            break
    
    return priceRange, zerosCounter, numberToRound

In [6]:
priceRange = None
df_list = []
typ = 'positionBook'

for i, response in enumerate(dataGenerator(typ, "2019-03-01T00:00:00Z", "2019-04-01T00:00:00Z")):
    if typ in response.keys():
        
        if not priceRange:      
            priceRange, zerosCounter, numberToRound = getInstrumentStats(response, typ)
                    
        row = processRecord(response, typ)
        df_list.append(row)
        
    else:
        print(response)
        
    break

In [ ]:
chuj = 0

for a in response['positionBook']['buckets']:
    chuj = chuj + float(a['longCountPercent']) + float(a['shortCountPercent'])
    
print(chuj)

In [ ]:

response['positionBook']['buckets']

In [60]:
df = pd.DataFrame(df_list, columns = [
        'time',
        'price',
        'roundedPrice',
        'level_0_l',
        'level_0_s',
        'level_1_up_l',
        'level_1_up_s',
        'level_1_down_l',
        'level_1_down_s',
        'level_2_up_l',
        'level_2_up_s',
        'level_2_down_l',
        'level_2_down_s',
        'level_3_up_l',
        'level_3_up_s',
        'level_3_down_l',
        'level_3_down_s',
        'level_4_up_l',
        'level_4_up_s',
        'level_4_down_l',
        'level_4_down_s',
        'level_5_up_l',
        'level_5_up_s',
        'level_5_down_l',
        'level_5_down_s',
    ])

In [61]:
df.to_csv('march.csv')

In [ ]:
data = {
        "from": "2019-01-01T00:00:00Z",
        "to": "2019-04-01T00:00:00Z",
        "granularity": "M5",
        "count": "5000",
    }

In [110]:
df_list = []

for candle in requests.get(
                    url = 'https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles?'+\
                    "from=2019-01-01T00:00:00Z"+\
                    "&to=2019-01-15T00:00:00Z"+\
                    "&granularity=M5"
                    ,
                    headers = {
                        'Authorization' : 'Bearer ' + key,
                    },

                ).json()['candles']:
    try:
        df_list.append([
            candle['time'],
            candle['volume'],
            candle['mid']['o'],
            candle['mid']['h'],
            candle['mid']['l'],
            candle['mid']['c'],
        ])
    except:
        print(candle)
        break
        
df = pd.DataFrame(df_list, columns=[
    'time',
    'volume',
    'open',
    'high',
    'low',
    'close',
])

In [120]:
orders_tmp = pd.read_csv('order_book/january.csv')

In [122]:
orders_tmp.head(1)

,Unnamed: 0,time,price,roundedPrice,level_0_l,level_0_s,level_1_up_l,level_1_up_s,level_1_down_l,level_1_down_s,...,level_3_down_l,level_3_down_s,level_4_up_l,level_4_up_s,level_4_down_l,level_4_down_s,level_5_up_l,level_5_up_s,level_5_down_l,level_5_down_s
0,0,2019-01-03T15:40:00Z,1.13998,1.14,0.0065,0.0978,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0,0,0,0,0,0,0


In [126]:
df.set_index(pd.to_datetime(df['time']))\
    .join(orders_tmp.set_index(pd.to_datetime(orders_tmp['time'])), rsuffix='dupa')\
    .dropna()\
    

,time,volume,open,high,low,close,Unnamed: 0,timedupa,price,roundedPrice,...,level_3_down_l,level_3_down_s,level_4_up_l,level_4_up_s,level_4_down_l,level_4_down_s,level_5_up_l,level_5_up_s,level_5_down_l,level_5_down_s
time,,,,,,,,,,,,,,,,,,,,,
2019-01-03 15:40:00+00:00,2019-01-03T15:40:00.000000000Z,508,1.14001,1.14047,1.13954,1.14029,0.0,2019-01-03T15:40:00Z,1.13998,1.1400,...,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-03 16:00:00+00:00,2019-01-03T16:00:00.000000000Z,462,1.13928,1.13942,1.13871,1.13912,1.0,2019-01-03T16:00:00Z,1.13931,1.1395,...,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-03 16:20:00+00:00,2019-01-03T16:20:00.000000000Z,564,1.13990,1.14112,1.13973,1.14050,2.0,2019-01-03T16:20:00Z,1.13987,1.1400,...,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-03 16:40:00+00:00,2019-01-03T16:40:00.000000000Z,437,1.13979,1.13994,1.13877,1.13931,3.0,2019-01-03T16:40:00Z,1.13976,1.1400,...,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-03 17:00:00+00:00,2019-01-03T17:00:00.000000000Z,297,1.13982,1.13988,1.13933,1.13969,4.0,2019-01-03T17:00:00Z,1.13987,1.1400,...,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-03 17:20:00+00:00,2019-01-03T17:20:00.000000000Z,198,1.13994,1.14031,1.13982,1.14002,5.0,2019-01-03T17:20:00Z,1.13990,1.1400,...,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-03 17:40:00+00:00,2019-01-03T17:40:00.000000000Z,151,1.14009,1.14040,1.14000,1.14011,6.0,2019-01-03T17:40:00Z,1.14006,1.1400,...,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-03 18:00:00+00:00,2019-01-03T18:00:00.000000000Z,148,1.13984,1.14020,1.13965,1.14018,7.0,2019-01-03T18:00:00Z,1.13988,1.1400,...,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-03 18:20:00+00:00,2019-01-03T18:20:00.000000000Z,141,1.14037,1.14050,1.14006,1.14014,8.0,2019-01-03T18:20:00Z,1.14034,1.1405,...,0.2149,1.4000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
def get_OrderBook(typ):

    date = datetime.strptime("2019-02-01T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%dT%H:%M:%SZ")
    priceRange = None
    df_list = []
    i = 0

    while True:
        i = i + 1

        try:
            response = requests.get(
                url = 'https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/'+typ,
                headers = {
                    'Authorization' : 'Bearer ' + key,
                },
                data = {
                    'time': date
                }
            ).json()
        except:
            print(date)

        date = (datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ") + timedelta(minutes=20)).strftime("%Y-%m-%dT%H:%M:%SZ")

        if typ in response.keys():

            if not priceRange:
                priceRange = response[typ]['bucketWidth']
                numberToRound = len(str(response[typ]['bucketWidth']).split('.')[1])
                
                zerosCounter = 0
                for number in str(priceRange).split('.')[1]:
                    if number == '0':
                        zerosCounter = zerosCounter + 1
                    else:
                        break
                
            price = float(response[typ]['price'])
            
            estimatePrices = []
            estimatePrices.append(float(str(round(price, zerosCounter))))
            estimatePrices.append(round(round(price, zerosCounter) - float(numberToRound / (10 ** (zerosCounter + 1))), zerosCounter + 1))
            estimatePrices.append(round(round(price, zerosCounter) + float(numberToRound / (10 ** (zerosCounter + 1))), zerosCounter + 1))
            
            diff = 100
            chosenMiddlePriceIndex = 0
            for i, p in enumerate(estimatePrices):
                if abs(price - p) < diff:
                    diff = abs(price - p)
                    chosenMiddlePriceIndex = i
            
            middlePrice = estimatePrices[chosenMiddlePriceIndex]

            tmpBuckets = {}
            for bucket in response[typ]['buckets']:
                tmpBuckets[float(bucket['price'])] = {
                    'l': bucket['longCountPercent'],
                    's': bucket['shortCountPercent'],
                }
            pricesKeys = list(tmpBuckets.keys())

            row = [
                response[typ]['time'],
                response[typ]['price'],
                middlePrice,
            ]

            if middlePrice in pricesKeys:
                row.append(tmpBuckets[middlePrice]['l'])
                row.append(tmpBuckets[middlePrice]['s'])
            else:
                row.append(0)
                row.append(0)

            UpPrice = middlePrice
            DownPrice = middlePrice
            floatedPriceBucket = float(priceRange)
            for i in range(5):
                UpPrice = UpPrice + floatedPriceBucket
                if UpPrice in pricesKeys:
                    row.append(tmpBuckets[UpPrice]['l'])
                    row.append(tmpBuckets[UpPrice]['s'])
                else:
                    row.append(0)
                    row.append(0)

                DownPrice = DownPrice + floatedPriceBucket
                if DownPrice in pricesKeys:
                    row.append(tmpBuckets[DownPrice]['l'])
                    row.append(tmpBuckets[DownPrice]['s'])
                else:
                    row.append(0)
                    row.append(0)

            df_list.append(row)

        if date == '2019-03-01T00:00:00Z':
            break
    
    df = pd.DataFrame(df_list, columns = [
        'time',
        'price',
        'roundedPrice',
        'level_0_l',
        'level_0_s',
        'level_1_up_l',
        'level_1_up_s',
        'level_1_down_l',
        'level_1_down_s',
        'level_2_up_l',
        'level_2_up_s',
        'level_2_down_l',
        'level_2_down_s',
        'level_3_up_l',
        'level_3_up_s',
        'level_3_down_l',
        'level_3_down_s',
        'level_4_up_l',
        'level_4_up_s',
        'level_4_down_l',
        'level_4_down_s',
        'level_5_up_l',
        'level_5_up_s',
        'level_5_down_l',
        'level_5_down_s',
    ])
    
    return df

In [ ]:
jan = get_OrderBook('positionBook')

2019-02-06T19:20:00Z


In [22]:
jan

,time,price,roundedPrice,level_0_l,level_0_s,level_1_up_l,level_1_up_s,level_1_down_l,level_1_down_s,level_2_up_l,...,level_3_down_l,level_3_down_s,level_4_up_l,level_4_up_s,level_4_down_l,level_4_down_s,level_5_up_l,level_5_up_s,level_5_down_l,level_5_down_s


In [17]:
jan.to_csv('march.csv')